In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import regex as re
import pandas as pd
import numpy as np

In [3]:
column_names = ["trained_on","type", "model", "setup", "run", "dataset", "split"]

directory = '../results/final1'

file_paths = [os.path.join(root, filename) for root, dirs, files in os.walk(directory) for filename in files]

result_dicts = []
re_pattern = r'[\:\n]'
for file_path in file_paths:
    if file_path.endswith(".txt"):
        file_path_split = file_path.split("/")
        file_parts =  file_path_split[3:-1]
        trained_on = file_parts[0]
        if trained_on in ["o", "lf"]:
            # print(len(column_names), len(file_parts))
            assert len(column_names) == len(file_parts)
            result_dict = {}
            for col_id,column_name in enumerate(column_names):
                result_dict[column_name] = file_parts[col_id]
            result_dicts.append(result_dict)
            with open(file_path, "r") as f:
                file_txt = f.read().strip()
                file_txt = re.split(re_pattern, file_txt)
                # print(file_txt)
                for i in range(0, len(file_txt), 2):
                    result_dict[file_txt[i].strip().replace('"','')] = float(file_txt[i+1].strip())
print(result_dicts)

[{'trained_on': 'o', 'type': 'sota', 'model': 'met_inf', 'setup': 'setup2', 'run': 'run3', 'dataset': 'fantasy', 'split': 'test', 'CONLL F1': 76.575020394785, 'Macro F1': 67.55232024980278, 'Micro F1': 69.93564102486842, 'Macro-Micro F1': 72.2316156773386}, {'trained_on': 'o', 'type': 'sota', 'model': 'met_inf', 'setup': 'setup2', 'run': 'run3', 'dataset': 'litbank', 'split': 'test', 'CONLL F1': 73.08116148236957, 'Macro F1': 61.78358622613669, 'Micro F1': 60.68614772880699, 'Macro-Micro F1': 59.395367337217564}, {'trained_on': 'o', 'type': 'sota', 'model': 'met_inf', 'setup': 'setup2', 'run': 'run1', 'dataset': 'fantasy', 'split': 'test', 'CONLL F1': 76.2539464609313, 'Macro F1': 65.95868018269468, 'Micro F1': 71.16333382507842, 'Macro-Micro F1': 73.1786012635418}, {'trained_on': 'o', 'type': 'sota', 'model': 'met_inf', 'setup': 'setup2', 'run': 'run1', 'dataset': 'litbank', 'split': 'test', 'CONLL F1': 71.95895222032559, 'Macro F1': 63.34439376396803, 'Micro F1': 60.39862795157311, '

In [4]:
result_df = pd.DataFrame(result_dicts)
# Define columns for summarization
columns_to_summarize = ["CONLL F1", "Macro F1", "Micro F1", "Macro-Micro F1"]

# Group by the specified columns and calculate mean, count, and standard deviation
summary = result_df.groupby(["setup","trained_on","dataset","split", "type", "model"])[columns_to_summarize].agg(['mean', 'count', 'std'])

summary['count'] = summary[('CONLL F1', 'count')]
summary.drop([(col, 'count') for col in columns_to_summarize ], axis=1, inplace=True)

# Calculate '(mean +- std)' for each metric
for col in columns_to_summarize:
    summary[(col, '')] = summary[(col, 'mean')].round(2).astype(str) + ' +- ' + summary[(col, 'std')].round(2).astype(str)

# Drop the original 'mean' and 'std' columns for each metric
summary.drop([ (col, 'mean') for col in columns_to_summarize ], axis=1, inplace=True)
summary.drop([ (col, 'std') for col in columns_to_summarize ], axis=1, inplace=True)

# Reset index to make the grouped columns as regular columns
# summary.reset_index(inplace=True)

summary_xlsx = summary.to_excel("../results/final2_summary_new_1.xlsx")